In [1]:
import numpy as np

In [181]:
class BinaryCrossEntropy:
    def __init__(self):
        pass

    def __call__(self, y_pred, y_true):
        ix_zeros = np.arange(0, y_true.shape[0])[y_true.reshape(-1) == 0]
        ix_ones = np.arange(0, y_true.shape[0])[y_true.reshape(-1) == 1]

        y_zero = np.log(1 - y_pred[ix_zeros] + 1e-10).sum()
        y_one = np.log(y_pred[ix_ones] + 1e-10).sum()

        return -1 * (y_zero + y_one)
    
    def grad_input(self, X, y_true):
        if y_true == 0:
            return -1/(1-X)
        else:
            return -1/X 
        


In [183]:
class Sigmoid:
    def __call__(self, X):
        return self.eval(X)
    
    def eval(self, X):
        return 1/(1+np.e**(-1*X))

    def grad_input(self, X):
        return self.eval(X)*(1 - self.eval(X))

class Dot:
    def __init__(self, input_size, units):
        self.W = np.random.randn(input_size, units)
        self.b = np.random.randn(units, 1)

    def __call__(self, X):
        return self.W.T.dot(X) + self.b

    def grad_w(self, X):
        I = np.identity(self.b.shape[0])
        return np.stack([I]*self.W.shape[1], axis=1)*X
    
    def grad_b(self):
        return np.identity(self.b.shape[0])

    def grad_input(self):
        return self.W.T

In [185]:
class Dense:

    def __init__(self, units, activation, input_size):
        self.units = units
        self.dot = Dot(input_size, units)
        self.activation = activation
        
    def eval(self, X):
        return self.activation(self.dot(X))

    def grad_parameters(self, X):
        da_dI = self.activation.grad_input(self.dot(X))
        dI_dw = self.dot.grad_w(X)
        da_dw = da_dI * dI_dw
        dI_db = self.dot.grad_b(X)
        da_db = da_dI * dI_db
        return (da_dw, da_db)
    
    def grad_input(self, X):
        g1 = self.activation.grad_input(self.dot(X))

        g2 = self.dot.grad_input()

        return g1.dot(g2)

In [196]:
class Sequential:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.outputs = []

    def add(self, layer):
        self.layers.append(layer)
        return self
    
    def forward_propagation(self, X):
        output = X.T
        for layer in self.layers:
            output = layer.eval(output)
        
        return output.T

    def _eval(self, X):
        return self.forward_propagation(X)
    
    def compile(self, loss):
        self.loss = loss

    def _eval_loss(self, X, y_true):
        if self.loss is None:
            raise RuntimeError("Model not compiled")
            
        return self.loss(self._eval(X), y_true)
    
    def backward_propagation(self):
        raise NotImplementedError("Backpropagation is not defined!")


In [197]:
model = Sequential()
model.add(Dense(units=2, activation=Sigmoid(), input_size=2))
model.add(Dense(units=2, activation=Sigmoid(), input_size=2))
model.add(Dense(units=1, activation=Sigmoid(), input_size=2))
model.compile(BinaryCrossEntropy())

In [217]:
X = np.random.randn(1, 2)
print("Y_pred", model._eval(X))
print("Loss", model._eval_loss(X, np.array([1])))

Y_pred [[0.72188274]]
Loss 0.3258925695278604
